<center><h1>Cleaning Strings and Dates in R</h1></center>
<center><h3>Ellen Duong</h3></center>
<center><h3>Paul Stey</h3></center>
<center><h3>2023-10-19</h3></center>

# 1. What is Data Cleaning?
  - No uniform definition "data cleaning"
  - Roughly speaking, refers to exploring the idiosyncrasies of a data set, and then addressing them in a principled manner so as to allow for data analysis

## 1.1 Examples of Data Cleaning

  - Recoding `"NULL"`, `" "`, `""`, to be `NA`
  - Eliminating duplicate entries
  - Ensure numeric data is being treated as numerics (e.g., `"2" + 2 != 4`)
  - Treating dates or timestamps as `Date` or `POSIXct` data type 

# 2. Cleaning Strings

  - Parsing/cleaning/extracting info from strings is extremely common
  - Parsing timestamp strings is a great example

## `stringr` package


In [2]:
# Load stringr package
library(stringr)

Common `stringr` functions

[Source with examples](https://appsilon.com/stringr-examples/)

| Function | Description | Format |
| --- | --- | --- |
| `str_length` | Returns the number of characters in a given string | `str_length(x)`
| `str_sub` | Returns a substring of a given string | `str_sub(x, start = <int: starting index>, end = <int: ending index>)` |
| `str_detect` | Returns a boolean or a vector of booleans that determin if the pattern exists in the given string. | `str_detect(x, <str: pattern>)` |
| `str_replace` | Replaces the first occurance of the pattern with the replacement string | `str_replace(x, <str: pattern>, <str: replacement string>)` |
| `str_replace_all` | Replaces all occurances of the pattern with the replacement string | `str_replace_all(x, <str: pattern>, <str: replacement string>)` |

| Function | Description | Format |
| --- | --- | --- |
| `str_count` | Counts the number of times a patter appears in a string | `str_count(x, <str: pattern>)` |
| `str_subset` | Returns a subset of vectors of strings that match a pattern | `str_subset(x, <str: pattern>)` |
| `str_trim` | Removes leading and trailing whitespaces | `str_trim(x)` |
| `str_split` | Splits a string or a vector of strings into a vector of substrings or a list of vectors of substrings, depending on the format of data passed in. | `str_split(x, <str: delimiter>)`

More common `stringr` functions

| Function | Description | Format |
| --- | --- | --- |
| `str_to_title` | Capitalizes the first letter of each word in a string | `str_to_title(x)` |
| `str_to_sentence` | Capitalizes the first letter of a string | `str_to_sentence(x)` |
| `str_to_upper` | Uppercase the entire string | `str_to_upper(x)` |
| `str_to_lower` | Lowercase the entire string | `str_to_lower(x)` |

## 2.1 Errors in our `officer_cnt`

In [1]:
# Load necessary packages and arrests data
library(stringr)
library(dplyr)

arrests_df <- read.csv("data/pvd_arrests_2021-10-03.csv")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
count_names <- function(names_str) {
    # This function should return the number of names in 
    # the string `names_str` that we pass to the function. 
    
    name_list <- str_split(names_str, ", ")
    name_vec <- unlist(name_list)
    k <- length(name_vec)
    
    return(k)
}

### 2.1.1 Inconsistencies in `arresting_officers` Column

In [3]:
head(arrests_df$arresting_officers, 10)

[1] " YGonzalez, LTaveras"     " NManfredi"              
 [3] " MPlace, JPerez, ASantos" " MPlace, JPerez, ASantos"
 [5] " MPlace, JPerez, ASantos" " MPlace, JPerez, ASantos"
 [7] " MPlace, JPerez, ASantos" " CVingi, SCooney"        
 [9] " CVingi, SCooney"         " CVingi, SCooney"

In [4]:
tail(arrests_df$arresting_officers, 10)

[1] "Dyer, Brian/ Georges, Wilkens"                                      
 [2] "Morgan, Glen/ Bass, Khari"                                          
 [3] "Levesque, Alexa/ Georges, Wilkens"                                  
 [4] "Bustamante, Genifer"                                                
 [5] "Colicci, Nathaniel/ Heeder, Jenna/ Prete, Louis/ Wagoner, Alexander"
 [6] "Colicci, Nathaniel/ Heeder, Jenna/ Prete, Louis/ Wagoner, Alexander"
 [7] "Colicci, Nathaniel/ Heeder, Jenna/ Prete, Louis/ Wagoner, Alexander"
 [8] "Colicci, Nathaniel/ Heeder, Jenna/ Prete, Louis/ Wagoner, Alexander"
 [9] "Silva, Eric"                                                        
[10] "Cooney, Sean"

## 2.2 Addressing the Inconsistency
  - Use different criteria for counting names with full-name format
    + Define function to identify full-name vs. first-initial format
    + Note: first-inital format always starts with two capital letters

In [5]:
LETTERS               # This is a built-in object in R

[1] "A" "B" "C" "D" "E" "F" "G" "H" "I" "J" "K" "L" "M" "N" "O" "P" "Q" "R" "S"
[20] "T" "U" "V" "W" "X" "Y" "Z"

In [6]:
"B" %in% LETTERS

[1] TRUE

## 2.3 Identifying Full-Name Format 
  - If the first two characters are uppercase, it's full-name format

In [7]:
is_uppercase <- function(chr) {
    res <- chr %in% LETTERS
    return(res)
}

has_full_names <- function(names_str) {
    char1 <- str_sub(names_str, 1, 1)
    char2 <- str_sub(names_str, 2, 2)
    
    res <- !(is_uppercase(char1) && is_uppercase(char2))
    return(res)
}

### 2.3.1 Testing our Functions

In [8]:
is_uppercase("a")                            # false
is_uppercase("b")                            # false
has_full_names("NManfredi")                  # Not full name
has_full_names("MPlace, JPerez, ASantos")    # Not full name

is_uppercase("A")
is_uppercase("B")
has_full_names("Newton, Frank")
has_full_names("Newton, Frank/ Chin, Rosemarie")

[1] FALSE

[1] FALSE

[1] FALSE

[1] FALSE

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

## 2.4 Fixing our `count_names()` Function

In [ ]:
old_count_names <- function(names_str) {
    name_list <- str_split(names_str, ", ")
    name_vec <- unlist(name_list)
    k <- length(name_vec)
    
    return(k)
}

In [ ]:
count_names <- function(names_str) {
    names_str_trm <- str_trim(names_str)     # remove whitespace
    
    if (has_full_names(names_str_trm)) {
        split_char <- "/ "
    } else {
        split_char <- ", "
    }
    
    name_list <- str_split(names_str_trm, split_char)
    name_vec <- unlist(name_list)
    k <- length(name_vec)
    
    return(k)
}

### 2.4.1 Testing New `count_names()`

In [ ]:
old_count_names("YGonzalez, LTaveras") == 2
old_count_names("Newton, Frank/ Chin, Rosemarie") == 2     # function is wrong
count_names("YGonzalez, LTaveras") == 2
count_names("Newton, Frank/ Chin, Rosemarie") == 2

## 2.5 Re-Counting Officers
  - Let's compare how the "old" (i.e., incorrect) method did relative to our new `count_names()`

In [ ]:
count_all_names <- function(col, old = FALSE) {

    n <- length(col)   # get the length of our input column
    cnts <- rep(0, n)  # allocate vector of zeros to populate with counts

    for (i in 1:n) {
        if (old) {
            cnts[i] <- old_count_names(col[i])
        } else {
            cnts[i] <- count_names(col[i])
        }
    }
    return(cnts) 
}

In [ ]:
arrests_df$old_officer_cnt <- count_all_names(arrests_df$arresting_officers, old = TRUE)

arrests_df$officer_cnt <- count_all_names(arrests_df$arresting_officers)

In [ ]:
head(arrests_df)

In [ ]:
tail(arrests_df, 12)

## 2.6 How Many Errors?

In [ ]:

sum(arrests_df$old_officer_cnt != arrests_df$officer_cnt)

In [ ]:
nrow(arrests_df)

<h1><center>Challenge Problem</center></h1>
    

Suppose that we have addresses in the form of strings such as `"123 Main St. Providence, RI, 02901"`. Now suppose that we want to write a function that gets the state abbreviation from that string, and returns that. Let's call our function `get_state_code()`. So, for example, from the string above, we would want our `get_state_code()` function to return `RI`.

As a hint, somewhere in our function we will want to use the `str_extract()` function to extract the following pattern `", ..,"`. This will match the state abbreviation, to get us started. We will also want to use the `str_sub()` function to extract only the characters that we want (i.e., the 3rd and 4th).

When our function is complete, the three tests below should print `TRUE`.

In [ ]:
print(get_state_code("123 Main St. Providence, RI, 02901") == "RI")
print(get_state_code("915 Adams Street South East Hutchinson, MN, 55350") == "MN")
print(get_state_code("1200 Pennsylvania Avenue NW. Washington, DC, 20004") == "DC")

<center><h1>Cleaning Dates in R</h1></center>

# 3. The _lubridate_ Package

  - Extremely powerful R package for working with dates and timestamps
  - Part of the _tidyverse_ family of packages (e.g., _dplyr_, _ggplot_, _stringr_)

In [ ]:
# load pacakges
# read in data

library(dplyr)
library(lubridate)

arrests_df <- read.csv("data/pvd_arrests_2021-10-03.csv")

## 3.1 Working with Timestamps
  - The _lubridate_ package has many built-in functions for timestamp data
  - Also often easily recognizes when a string _is_ a timestamp

In [ ]:
ts <- "2020-10-11 02:30:59"     # ISO 8601 format: YYYY-MM-DD HH:MM:SS

year(ts)                        

In [ ]:
month(ts)

In [ ]:
day(ts)

### 3.1.1 Extracting Time

In [ ]:
ts <- "2020-10-11 02:30:59"

hour(ts)
minute(ts)
second(ts)

In [ ]:
am(ts)             # is it AM time (i.e., morning)?

dst(ts)

### 3.1.2 Extracting Day-of-Week

In [ ]:
ts <- "2020-10-11 02:30:59"

wday(ts)

In [ ]:
toString(wday(ts, label = TRUE))

## 3.2 Other Timestamp Formats

In [ ]:
ts2 <- "2020-10-11"

toString(wday(ts2, label = TRUE))

In [ ]:
ts3 <- as_datetime("20201011")

toString(wday(ts3, label = TRUE))

### 3.2.1 Non ISO 8601 Format
  - We can also tell _lubridate_ package how to parse non-obvious timestamps

In [ ]:
ts3 <- "October 11, 2020"

month(ts3)             

In [ ]:
mdy(ts3)             # Month-day-year format (also dmy(), ymd(), and others)

In [ ]:
month(mdy(ts3))

# 4. Math with Dates and Timestamps

  - The _lubridate_ package also makes it easy to do math with dates and times

In [ ]:
end_of_october <- as_date("2020-10-31")

In [ ]:
end_of_october + days(1)

In [ ]:
time1 <- as_datetime("2020-10-11 03:45:52")
time2 <- as_datetime("2020-10-13 23:41:09")

time2 - time1

## 4.1 Date/Time Intervals

In [ ]:
time1 <- as_datetime("2020-10-12")
time2 <- as_datetime("2020-10-15")


dt_intr <- interval(time1, time2)

In [ ]:
as_datetime("2020-10-13") %within% dt_intr

In [ ]:
now() %within% dt_intr

<h1><center>Challenge Problem</h1></center>


Let's write a function called `is_within_range()` that takes the following three arguement: `start_day`, `end_day`, and `test_day`. These three arguments will be dates in the form of strings. The function will return a boolean value, indicating whether or not `test_day` is within the range from `start_day` to `end_day`. 

Three points are worth noting. First, we will need to use the `as_date()` function to "cast" the string form of the arguments in to the date type. And second, we'll want to use the `interval()` function to take the beginning and ending dates an create an interval range. Finally, we will almost certainly want to use the `%within%` operator to check wether our test day is within our range.

When our function is working, the test below should all print `TRUE`.

In [ ]:
print(is_within_range("2021-09-10", "2021-11-10", "2021-10-10"))
print(is_within_range("2021-09-10", "2022-05-03", "2021-10-10"))
print(is_within_range("2011-09-10", "2020-01-10", "2021-12-08") == FALSE)

# 5. Arrests by Day-of-Week

  - Suppose we want to explore the number of arrests by the day of the week
  

## 5.1 Create `day_of_week()` Function

In [ ]:

day_of_week <- function(timestamps) {
    
    n <- length(timestamps)  # get length of input column
    day <- rep("", n)        # allocate vector for day of week
    
    # iterate over elements of input column and return 
    # the day of the week for each timestamp
    
    for (i in 1:n) {
        day[i] <- toString(wday(timestamps[i], label = TRUE))
    }
    return(day)
}


### 5.1.1 Creating `weekday` Column
   - Now we can use our newly created `day_of_week()` function to add a new column

In [ ]:
# use out `day_of_week()` function to create new column
# in our original dataframe

arrests_df$weekday <- day_of_week(arrests_df$arrest_date)

In [ ]:
# use head() to examine updated dataframe

head(arrests_df)

### 5.1.2 Counts by `weekday`

We can now obtain the counts by day of the week using the `table()` function. We simply pass it the column of the dataframe for which we want to create a tabular summary.

In [ ]:
# use table() to get counts of arrests by `weekday`

table(arrests_df$weekday)